# Measure densities of different cell types

This example demonstrates how to classify nuclei into one of three classes and calculate the density of each type of nuclei in every ROI marked in an image. 

### Import dependencies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
pd.set_option('display.width', 2000)
from utils import readImage
import numpy as np
from sklearn.cluster import KMeans

### Read dataframes

In [ ]:

nucleus_df = pd.read_csv('nuclei.csv')
image_df = pd.read_csv('images.csv')

### Clustering
Cluster nuclei into one of three clusters based on Ch1Intensity

In [ ]:
channel_key = 'Ch1Intensity'
nucleus_df['Cluster_Label'] = np.nan
num_clusters = 3

for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    intensity_values = df[channel_key].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init="auto")
    clusters = kmeans.fit_predict(intensity_values)
    
    sorted_cluster_indices = np.argsort(kmeans.cluster_centers_.flatten())
    cluster_mapping = {old_label: new_label for new_label, old_label in enumerate(sorted_cluster_indices)}
    
    cluster_labels = np.array([cluster_mapping[label] for label in clusters])
    nucleus_df.loc[df.index, 'Cluster_Label'] = cluster_labels

# Assign cell types based on the new consistent cluster labels
nucleus_df.loc[nucleus_df['Cluster_Label'] == 2.0, 'CellType'] = 'Neuron'
nucleus_df.loc[nucleus_df['Cluster_Label'] == 1.0, 'CellType'] = 'NeuronLow'
nucleus_df.loc[nucleus_df['Cluster_Label'] == 0.0, 'CellType'] = 'Undefined'

# Print the counts of each cell type

# Drop the Cluster_Label column
nucleus_df.drop(columns=['Cluster_Label'], inplace=True)

### Calculate densities

- Filter out nuclei outside of the regions of interest and merge neuron and image dataframes.
- Calculate densities

In [ ]:
regions_df = nucleus_df[nucleus_df['Location'] != 'Undefined']
neuron_counts = regions_df.groupby(['Location', 'Condition', 'ImageName', 'CellType']).size().reset_index(name='Nuclei')
merged_df = pd.merge(neuron_counts, image_df, on='ImageName')

# Calculate neuron density for each region
merged_df['NeuronDensity'] = merged_df['Nuclei']  # Initialize the column with Nuclei counts
merged_df.loc[merged_df['Location'] == 'CA1', 'NeuronDensity'] /= merged_df['CA1Volume']  # Divide by CA1Volume where Location is CA1
merged_df.loc[merged_df['Location'] == 'CA3', 'NeuronDensity'] /= merged_df['CA3Volume']  # Divide by CA3Volume where Location is CA3
merged_df.loc[merged_df['Location'] == 'DG', 'NeuronDensity'] /= merged_df['DGVolume']    # Divide by DGVolume where Location is DG
merged_df['NeuronDensity'] *= 10**6

print(merged_df.head())


### Visualize results

Visualize first as a table

In [ ]:
#print(merged_df[(merged_df['CellType'] != 'Undefined') & (merged_df['Location'] == 'DG')])

table_df = merged_df.groupby(['Location', 'Condition_x', 'CellType']).agg({
    'NeuronDensity': ['mean', 'std']
})
print(table_df)

Then as a set of boxplots

In [ ]:
palette = sns.color_palette(['royalblue'], 2)
for location in merged_df['Location'].unique():
    # Filter dataframe for the current location
    subset_df = merged_df[merged_df['Location'] == location]
    x_order = ['Undefined', 'NeuronLow', 'Neuron']
    hue_order = ['Sham', 'Contra', 'Ipsi']
    
    # Create the boxplot
    plt.figure(figsize=(8, 6))  # Adjust size if needed
    boxplot = sns.boxplot(data=subset_df, x='CellType', y='NeuronDensity', showfliers=False, hue='Condition_x', hue_order=hue_order, order=x_order, palette='Set3')
    stripplot = sns.stripplot(data=subset_df, x='CellType', y='NeuronDensity',order = x_order, hue='Condition_x', hue_order=hue_order, dodge=True, palette=palette, alpha=0.7)
    # Set labels and title
    plt.xlabel('')
    plt.ylabel('Nuclei / 100 \u03bcm^3')
    #plt.title(location)
    
    # Rotate x-axis labels for better visibility if needed
    #plt.xticks(rotation=45)
    
    handles, labels = boxplot.get_legend_handles_labels()

    # Create custom legend
    plt.legend(handles=handles[:3], labels=labels[:3])
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    name = f"plots/{location}_neurons_volume_v2.pdf"
    #plt.savefig(name)
    plt.show()

### Statistical analysis

Conduct a one way ANOVA for each hippocampal region followed by post-hoc testing

In [ ]:
from scipy.stats import f_oneway
from scipy.stats import friedmanchisquare

anova_results = {}
tukey_results = {}

for region in merged_df['Location'].unique():
    for cell_type in merged_df['CellType'].unique():
        type_data = [merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Condition_x'] == condition) & (merged_df['Location'] == region)]['NeuronDensity'] for condition in merged_df['Condition_x'].unique()]
        anova_results[cell_type] = f_oneway(*type_data)
        
        print(f"ANOVA results for {cell_type} in {region}: {anova_results[cell_type]}")
        
        # Perform Tukey's HSD test for each class of neuron
        mc = MultiComparison(merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Location'] == region)]['NeuronDensity'], merged_df[(merged_df['CellType'] == cell_type) & (merged_df['Location'] == region)]['Condition_x'])
        tukey_results[cell_type] = mc.tukeyhsd()
        print(f"Tukey's HSD results for {cell_type} in {region}:\n{tukey_results[cell_type]}")

